# Entrenamiento de un modelo TransE con `new_triplets.csv`

A continuación se muestra un ejemplo completo en Python (para ser ejecutado en un Notebook o script) que:

1. Carga las tripletas del archivo `new_triplets.csv` (generado previamente).
2. Divide el dataset en conjuntos de entrenamiento y test.
3. Crea un modelo TransE usando la clase `ScoringBasedEmbeddingModel` de AmpliGraph con el parámetro `scoring_type="TransE"`.
4. Compila y entrena el modelo.
5. Evalúa el modelo usando métricas de link prediction (por ejemplo, el MRR).


In [25]:
import pandas as pd
from ampligraph.datasets import load_from_csv
from ampligraph.evaluation import train_test_split_no_unseen, mrr_score, mr_score, hits_at_n_score
from ampligraph.latent_features.models import ScoringBasedEmbeddingModel
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow as tf

from ampligraph.latent_features.loss_functions import PairwiseLoss

from ampligraph.evaluation import train_test_split_no_unseen, mrr_score, mr_score, hits_at_n_score
from ampligraph.latent_features.models import ScoringBasedEmbeddingModel


In [26]:
# 1. Cargar el CSV completo de new_triplets.csv
df = pd.read_csv("new_triplets.csv")
print("Total de tripletas en el CSV original:", len(df))

Total de tripletas en el CSV original: 3718024


/tmp/ipykernel_7098/352491218.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("new_triplets.csv")


In [27]:
# 2. Seleccionar aleatoriamente 10,000 filas
# Si el CSV tiene menos de 10,000 filas, tomará todas.
n_muestras = min(len(df), 10000)
df_sample = df.sample(n=n_muestras, random_state=42)
print("Total de tripletas tras muestreo aleatorio:", len(df_sample))

Total de tripletas tras muestreo aleatorio: 10000


In [28]:
# 3. Convertir las columnas a str para evitar conflictos de tipos
df_sample['subject'] = df_sample['subject'].astype(str)
df_sample['relation'] = df_sample['relation'].astype(str)
df_sample['object'] = df_sample['object'].astype(str)

# 3.1 Convertir a array de tripletas
triples = df_sample[['subject', 'relation', 'object']].values

In [29]:
# 4. Dividir en train/test (permitiendo duplicación para evitar entidades unseen)
train_triples, test_triples = train_test_split_no_unseen(
    triples,
    test_size=0.1,
    seed=42,
    allow_duplication=True
)
print("Train size:", len(train_triples))
print("Test size:", len(test_triples))

Train size: 9775
Test size: 1000


# Configuración de Hiperparámetros Sugerida para un Dataset de ~3.7M de Tripletas

A continuación se propone una configuración de hiperparámetros como punto de partida para entrenar un modelo TransE con AmpliGraph en un dataset de aproximadamente 3.7 millones de tripletas.

- **η (eta) = 10:** Número estándar de muestras negativas por triple.
- **k = 200:** Aumentar la dimensión de los embeddings (de 100 a 200) puede ayudar a capturar mayor complejidad en un grafo grande, aunque aumenta el coste computacional.
- **Batch size = 2000:** Usar un batch size mayor reduce el número de pasos por época y puede acelerar el entrenamiento, siempre que la memoria lo permita.
- **Learning rate = 1e-4:** Una tasa de aprendizaje más baja puede estabilizar el entrenamiento en CPU, evitando oscilaciones en el loss.


In [30]:
# 5. Crear y compilar el modelo TransE con hiperparámetros ajustados
# Ajustes para mejorar precisión:
# - Incrementamos k de 100 a 200 para mayor capacidad
# - Reducimos el learning rate de 1e-3 a 1e-4 para actualizaciones más suaves
# - Aumentamos las épocas a 200 y usamos EarlyStopping
model = ScoringBasedEmbeddingModel(
    eta=10,              # Número de muestras negativas por triple
    k=200,               # Dimensión de embeddings aumentada
    scoring_type="ComplEx",
    seed=0
)

# Usar PairwiseLoss con margen=1
loss_obj = PairwiseLoss(loss_params={'margin': 1})

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=loss_obj,
    run_eagerly=True  # Ejecuta en modo eager para facilitar la depuración
)

In [31]:
# 6. Entrenar el modelo con EarlyStopping para evitar overfitting
history = model.fit(
    train_triples,
    batch_size=2000,     # Batch size mayor para acelerar el entrenamiento
    epochs=300,          # Entrenar por 200 épocas (EarlyStopping se encargará de parar si no mejora)
    verbose=True,
    callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=10,
        restore_best_weights=True
    )]
)

Epoch 1/300
6/6 [==============================] - 1s 95ms/step - loss: 19550.0156
Epoch 2/300
6/6 [==============================] - 0s 81ms/step - loss: 19548.7695
Epoch 3/300
6/6 [==============================] - 0s 76ms/step - loss: 19547.6113
Epoch 4/300
6/6 [==============================] - 0s 77ms/step - loss: 19546.4668
Epoch 5/300
6/6 [==============================] - 0s 76ms/step - loss: 19545.3320
Epoch 6/300
6/6 [==============================] - 0s 76ms/step - loss: 19544.1914
Epoch 7/300
6/6 [==============================] - 0s 79ms/step - loss: 19543.0391
Epoch 8/300
6/6 [==============================] - 0s 80ms/step - loss: 19541.8633
Epoch 9/300
6/6 [==============================] - 0s 82ms/step - loss: 19540.6699
Epoch 10/300
6/6 [==============================] - 0s 79ms/step - loss: 19539.4434
Epoch 11/300
6/6 [==============================] - 0s 79ms/step - loss: 19538.1934
Epoch 12/300
6/6 [==============================] - 0s 78ms/step - loss: 19536.9082
E

In [18]:
# 7. Evaluar el modelo (se evalúa corrompiendo sujeto y objeto por separado)
ranks_so = model.evaluate(
    test_triples,
    batch_size=100,
    corrupt_side='s+o'
)

# Calcular métricas
mrr_so = mrr_score(ranks_so)
mr_so = mr_score(ranks_so)
hits1_so = hits_at_n_score(ranks_so, 1)
hits10_so = hits_at_n_score(ranks_so, 10)
hits100_so = hits_at_n_score(ranks_so, 100)


print("=== Evaluación TransE - Corrupción de S + O ===")
print(f"MRR(s): {mrr_so:.4f}")
print(f"MR(s): {mr_so:.2f}")
print(f"Hits@1(s): {hits1_so:.4f}")
print(f"Hits@10(s): {hits10_so:.4f}")
print(f"Hits@100(s): {hits100_so:.4f}")


11/11 [==============================] - 2s 173ms/step
=== Evaluación TransE - Corrupción de S + O ===
MRR(s): 0.0004
MR(s): 5930.03
Hits@1(s): 0.0000
Hits@10(s): 0.0000
Hits@100(s): 0.0000


# Resumen del Entrenamiento y Evaluación del Modelo TransE

## Configuración y Entrenamiento
- **Dataset:** Se usaron 100,000 muestras (tripletas) extraídas aleatoriamente de `new_triplets.csv`.
- **Modelo:** Se entrenó un modelo TransE usando la clase `ScoringBasedEmbeddingModel`.
- **Hiperparámetros:**
  - **Número de muestras negativas (η):** 10
  - **Dimensión de los embeddings (k):** 200
  - **Learning Rate:** 1e-4 (para actualizaciones más suaves)
  - **Batch Size:** 2000
  - **Épocas:** 200 (con EarlyStopping implementado, aunque se completaron todas las épocas en este resumen)
- **Pérdida (loss):**
  - Inicialmente, el loss era de aproximadamente 17,922.
  - Tras 200 épocas, el loss descendió a alrededor de 1,113.
  - La curva de pérdida mostró una disminución constante, lo que indica que el modelo está aprendiendo la tarea de embebido de relaciones.

## Evaluación del Modelo

Se evaluó el modelo separadamente en dos escenarios:
  
### Corrupción de Sujeto
- **MRR (Mean Reciprocal Rank):** 0.0001  
- **MR (Mean Rank):** 43,610.73  
- **Hits@1:** 0.0000  
- **Hits@10:** 0.0000  
- **Hits@100:** 0.0002  

### Corrupción de Objeto
- **MRR (Mean Reciprocal Rank):** 0.0585  
- **MR (Mean Rank):** 1,650.38  
- **Hits@1:** 0.0000  
- **Hits@10:** 0.1932  
- **Hits@100:** 0.4782  

### Métrica Promedio
- **Promedio MRR (sujeto + objeto):** 0.0293

## Interpretación y Conclusiones

- **Reducción del Loss:**  
  La pérdida disminuyó significativamente (de ~17,922 a ~1,113) a lo largo de 200 épocas, lo que es una señal positiva de que el modelo está aprendiendo a diferenciar entre tripletas positivas y negativas.

- **Desempeño en la Predicción de Entidades:**  
  - **Corrupción de Sujeto:**  
    El MRR es prácticamente 0 y el Mean Rank es muy alto (~43,610), lo que sugiere que el modelo tiene serias dificultades para predecir correctamente el sujeto.  
  - **Corrupción de Objeto:**  
    El MRR es algo mayor (0.0585) y el Mean Rank es considerablemente menor (~1,650), lo que indica que el modelo predice algo mejor el objeto de la tripleta.  
  - La diferencia entre ambos lados sugiere que el modelo está aprendiendo de forma desigual, con una predicción muy pobre en el lado del sujeto.

- **Baja Precisión General:**  
  Un promedio MRR de 0.0293 es bajo, lo que implica que, en promedio, la entidad correcta aparece muy abajo en el ranking de predicción. Además, los valores de Hits@N son muy bajos, especialmente en Hits@1 y Hits@10, lo que indica que el modelo rara vez posiciona correctamente la entidad en las primeras posiciones.

## Sugerencias para Mejorar el Modelo

1. **Aumentar la Dimensión de los Embeddings:**  
   Probar con un valor mayor a `k=200` para capturar representaciones más ricas del grafo.

2. **Ajustar la Tasa de Aprendizaje:**  
   Considerar un `learning_rate` aún más bajo (por ejemplo, 1e-5 o 1e-4) para permitir actualizaciones más finas, especialmente si se observa que el loss se estabiliza sin bajar lo suficiente.

3. **Aumentar el Número de Épocas:**  
   Si los recursos lo permiten, entrenar por más épocas podría ayudar, siempre vigilando las métricas en un conjunto de validación para evitar overfitting.

4. **Revisar la Distribución de Datos:**  
   Analizar la frecuencia de aparición de entidades y relaciones. Si existen muchas entidades con baja frecuencia, esto podría dificultar la generalización del modelo. Considerar técnicas de filtrado o muestreo equilibrado.

5. **Explorar Otros Modelos:**  
   Dada la asimetría en la predicción (sujeto vs. objeto), probar otros modelos de embeddings (como ComplEx, DistMult o RotatE) podría ofrecer un desempeño mejor en ciertas tareas de link prediction.
